In [1]:
%cd drive/Shareddrives/CS704_Project/llm_based_modeling/
!ls

/content/drive/Shareddrives/CS704_Project/llm_based_modeling
processed_data_3_majority.csv  qllama2_topic_modelling.ipynb
processed_data_5_majority.csv  topic_generation.ipynb


In [2]:
!pip install --upgrade pip
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor
!pip install openai==0.28
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
from collections import defaultdict
from datetime import datetime
import pandas as pd
import json
import numpy as np
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from google.colab import userdata

In [4]:
openai.api_key = userdata.get('OPENAI_API_KEY')

In [5]:
df = pd.read_csv("processed_data_3_majority.csv")
df.head(10)

<ipython-input-5-f12c8345a5b7>:1: DtypeWarning: Columns (0,1,2,3,4,5,11,12,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("processed_data_3_majority.csv")


,tweet,target,manual_keywords,statement,3_label_majority_answer,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,cleaned_tweet,Truthfulness
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,4262.0,3619.0,34945.0,16423.0,44.0,...,5.0,0.0,1.0,0.0,33.0,3.0,5.0,19.0,Biden Blunders 6 Month Update Inflation Delt...,True
1,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,1393.0,1621.0,31436.0,37184.0,64.0,...,1.0,0.0,0.0,0.0,14.0,0.0,2.0,34.0,Not as many people are literally starving and ...,True
2,THE SUPREME COURT is siding with super rich pr...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,9.0,84.0,219.0,1184.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,4.0,10.0,THE SUPREME COURT is siding with super rich pr...,True
3,@POTUS Biden Blunders\n\nBroken campaign promi...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,4262.0,3619.0,34945.0,16423.0,44.0,...,3.0,0.0,0.0,1.0,6.0,8.0,1.0,30.0,Biden Blunders Broken campaign promises Infla...,True
4,@OhComfy I agree. The confluence of events rig...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,70.0,166.0,15282.0,2194.0,0.0,...,3.0,0.0,1.0,0.0,11.0,3.0,2.0,19.0,I agree The confluence of events right now is ...,True
5,"I've said this before, but it really is incred...",True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,29010.0,5081.0,120924.0,89474.0,405.0,...,2.0,0.0,0.0,1.0,8.0,0.0,1.0,18.0,Ive said this before but it really is incredib...,True
6,"As many face backlogged rent payments, America...",True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,33.0,21.0,432.0,62.0,0.0,...,2.0,0.0,0.0,0.0,9.0,0.0,2.0,24.0,As many face backlogged rent payments American...,True
7,@Thomas1774Paine @JoeBiden\n#DOJ@TheJusticeDep...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,919.0,620.0,131682.0,216319.0,71.0,...,2.0,0.0,0.0,0.0,11.0,3.0,2.0,19.0,INSTRUCTING SUPREMECOURT DONOTBLOCK NEWEVIC...,True
8,@SocialismIsDone @TheeKHiveQueenB Its a win fo...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,18002.0,15754.0,969539.0,497128.0,48.0,...,3.0,0.0,1.0,0.0,7.0,0.0,1.0,28.0,Its a win for AmericansIm not worried about ta...,True
9,@daysofarelives2 @Sen_JoeManchin There is not ...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,0.0,0.0,2.0,132.0,0.0,...,1.0,0.0,0.0,0.0,5.0,0.0,1.0,20.0,There is not and never has been stimulus check...,True


In [6]:
df['Truthfulness'].value_counts()

True     67656
False    65176
Cc        6839
True      1317
False       35
Name: Truthfulness, dtype: int64

In [12]:
unique_df = df.drop_duplicates(subset='statement', keep='first').reset_index(drop=True)
print(unique_df.shape)
unique_df.head()

(1060, 66)


,tweet,target,manual_keywords,statement,3_label_majority_answer,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,cleaned_tweet,Truthfulness
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,4262.0,3619.0,34945.0,16423.0,44.0,...,5.0,0.0,1.0,0.0,33.0,3.0,5.0,19.0,Biden Blunders 6 Month Update Inflation Delt...,True
1,@alwaysflexing @CNNPolitics @cnni @POTUS I kno...,True,"Trump administration, 5000,taliban","The Trump administration worked to free 5,000 ...",Agree,1130.0,1786.0,1512.0,56356.0,41.0,...,0.0,0.0,0.0,0.0,4.0,0.0,4.0,16.0,I know But it took a lot of bravery for Trump ...,True
2,US Deep State Gestapo leaves 100 billion dolla...,True,"afghanistan, 100 billion dollars","In Afghanistan, over 100 billion dollars spent...",Agree,49.0,86.0,338.0,2035.0,0.0,...,2.0,0.0,3.0,0.0,10.0,4.0,3.0,29.0,US Deep State Gestapo leaves 100 billion dolla...,True
3,A photo showing COVID-19 patients lying on the...,True,"Florida, COVID-19, patients, floor",A photo shows two COVID-19 patients lying on t...,Agree,9.0,215.0,11.0,3845.0,0.0,...,3.0,0.0,1.0,0.0,22.0,3.0,4.0,14.0,A photo showing COVID19 patients lying on the ...,True
4,@daemonx11 @DaveStephens124 @ImKnotTheOne @Tom...,True,"50 years, minimum wage, inflation",Its been over 50 years since minimum (wage) an...,Agree,26.0,128.0,746.0,2429.0,0.0,...,4.0,0.0,1.0,0.0,32.0,9.0,2.0,41.0,I suspect dr Dave has no idea what hes talking...,True


In [14]:
unique_df = unique_df[unique_df['Truthfulness'].isin([True, False])]
print(unique_df.shape)

(1039, 66)


In [15]:
unique_df['Truthfulness'].value_counts()

True     556
False    483
Name: Truthfulness, dtype: int64

In [20]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
unique_df = unique_df[['statement', 'cleaned_tweet', 'Truthfulness']].reset_index()
print(unique_df.shape)
X = unique_df.drop('Truthfulness', axis=1)  # Features
y = unique_df['Truthfulness']  # Target variable

# Create an instance of StratifiedShuffleSplit
stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

# Use the splitter to generate train and test indices
for train_index, test_index in stratified_splitter.split(X, y):
    train_data = unique_df.loc[train_index]
    test_data = unique_df.loc[test_index]

# Display the train and test sets
print("Train Set:")
print(train_data.shape)
print("\nTest Set:")
print(test_data.shape)

(1039, 4)
Train Set:
(935, 4)

Test Set:
(104, 4)


In [24]:
test_data = test_data.reset_index()
test_data.head()

,level_0,index,statement,cleaned_tweet,Truthfulness
0,648,669,Statistics show thatmore people of color don't...,Dan PatrickIf theyre worried about people of c...,False
1,825,846,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False
2,1037,1058,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False
3,819,840,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False
4,687,708,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False


In [26]:
test_data = test_data.drop(['level_0', 'index'], axis = 1)
test_data.head()

,statement,cleaned_tweet,Truthfulness
0,Statistics show thatmore people of color don't...,Dan PatrickIf theyre worried about people of c...,False
1,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False
2,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False
3,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False
4,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False


In [32]:
test_data.to_csv("test_data.csv", index = False)

In [22]:
test_data['Truthfulness'].value_counts()

True     56
False    48
Name: Truthfulness, dtype: int64

# Topic Modelling

In [33]:
test_data = pd.read_csv("test_data.csv")
print(test_data.shape)
print(test_data['Truthfulness'].value_counts())
test_data.head()

(104, 3)
True     56
False    48
Name: Truthfulness, dtype: int64


,statement,cleaned_tweet,Truthfulness
0,Statistics show thatmore people of color don't...,Dan PatrickIf theyre worried about people of c...,False
1,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False
2,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False
3,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False
4,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False


In [34]:
test_data['cleaned_tweet'][0]

'Dan PatrickIf theyre worried about people of color  on the Democrats side who came up with this drivein voting  statistics show thatmore people of color dont have cars than notSo how dodrivethru voting centershelp those folks PolitiFacts rulingFalse'

In [28]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the text provided. List 3 to 5 most important topics or keywords discussed in the following sentences in the format [keyword1, keyword2, keyword3]- "
    },
    {
      "role": "user",
      "content": "Biden Blunders  6 Month Update  Inflation Delta mismanagement COVID for kids Abandoning Americans in Afghanistan Arming the Taliban S Border crisis Breaking job growth Abuse of power Many Exec Orders 35T through Reconciliation Eviction Moratoriumwhat did I miss"
    }
  ],
  temperature=0.3,
  max_tokens=256,
  top_p=0.3,
  frequency_penalty=1,
  presence_penalty=1
)

In [29]:
response["choices"][0]["message"]["content"].strip()

'Biden Blunders, Inflation, Delta mismanagement, COVID for kids, Abandoning Americans in Afghanistan'

In [30]:
system_content = "You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the text provided. List 3 to 5 most important topics or keywords discussed in the following sentences in the format [keyword1, keyword2, keyword3]- "

In [35]:
tweet = test_data['cleaned_tweet'][0]
print(tweet)
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": system_content
    },
    {
      "role": "user",
      "content": tweet
    }
  ],
  temperature=0.3,
  max_tokens=256,
  top_p=0.3,
  frequency_penalty=1,
  presence_penalty=1
)
print(response["choices"][0]["message"]["content"].strip())

Dan PatrickIf theyre worried about people of color  on the Democrats side who came up with this drivein voting  statistics show thatmore people of color dont have cars than notSo how dodrivethru voting centershelp those folks PolitiFacts rulingFalse
Dan Patrick, drive-in voting, people of color, statistics, PolitiFact ruling


In [36]:
content_list = list(test_data['cleaned_tweet'])
print(len(content_list))

104


In [37]:
import asyncio
import aiohttp
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai.api_key
}

system_prompt =  "You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the text provided. List 3 to 5 most important topics or keywords discussed in the following sentences in the format [keyword1, keyword2, keyword3]- "

class ProgressLog:
    def __init__(self, total):
        self.total = total
        self.done = 0

    def increment(self):
        self.done = self.done + 1

    def __repr__(self):
        return f"Done runs {self.done}/{self.total}."

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(20), before_sleep=print, retry_error_callback=lambda _: None)
async def get_completion(content, session, semaphore, progress_log):
    async with semaphore:

        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json={
            "model": "gpt-3.5-turbo",
            "messages": [
                {
                  "role": "system",
                  "content": system_prompt
                },
                {
                  "role": "user",
                  "content": content
                }
              ],
            "temperature": 0.1,
            "max_tokens": 64,
            "top_p": 0.3,
            "frequency_penalty": 1.0,
            "presence_penalty": 1.0
        }) as resp:

            response_json = await resp.json()

            progress_log.increment()
            print(progress_log)

            return response_json["choices"][0]['message']["content"]

async def get_completion_list(content_list, max_parallel_calls, timeout):
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    progress_log = ProgressLog(len(content_list))

    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(timeout)) as session:
        return await asyncio.gather(*[get_completion(content, session, semaphore, progress_log) for content in content_list])

In [38]:
max_parallel_calls = 20
timeout = 200
generated_topics = await get_completion_list(content_list, max_parallel_calls, timeout)

Done runs 1/104.
Done runs 2/104.
Done runs 3/104.
Done runs 4/104.
Done runs 5/104.
Done runs 6/104.
Done runs 7/104.
Done runs 8/104.
Done runs 9/104.
Done runs 10/104.
Done runs 11/104.
Done runs 12/104.
Done runs 13/104.
Done runs 14/104.
Done runs 15/104.
Done runs 16/104.
Done runs 17/104.
Done runs 18/104.
Done runs 19/104.
Done runs 20/104.
Done runs 21/104.
Done runs 22/104.
Done runs 23/104.
Done runs 24/104.
Done runs 25/104.
Done runs 26/104.
Done runs 27/104.
Done runs 28/104.
Done runs 29/104.
Done runs 30/104.
Done runs 31/104.
Done runs 32/104.
Done runs 33/104.
Done runs 34/104.
Done runs 35/104.
Done runs 36/104.
Done runs 37/104.
Done runs 38/104.
Done runs 39/104.
Done runs 40/104.
Done runs 41/104.
Done runs 42/104.
Done runs 43/104.
Done runs 44/104.
Done runs 45/104.
Done runs 46/104.
Done runs 47/104.
Done runs 48/104.
Done runs 49/104.
Done runs 50/104.
Done runs 51/104.
Done runs 52/104.
Done runs 53/104.
Done runs 54/104.
Done runs 55/104.
Done runs 56/104.
D

In [46]:
len(generated_topics)
print(generated_topics)

['Dan Patrick, drive-in voting, people of color, statistics, PolitiFact ruling', 'Biden speech, unvaccinated, lepers, missing relative at the dinner table, plagiarized.', 'face covering, concealed weapon', "[formal challenge, Georgia's electoral votes, Kelly Loeffler, House Republicans, riots]", 'Stand Up Fight Back Atlanta rally, anti-Asian violence, spa shooting victims, China', 'Trump, poverty rate, Wisconsin families, lowest rate in 22 years, unemployment rate, historic low, economy, USA chant', 'Trump, middle class, corporate cut, deceit and lies, Terrible Ted Cruz', 'VP of Academic Affairs, nursing home, HR friend in Florida', 'Ronald Reagan, amnesty, savings and loan industry collapse, taxes, arms sales to Iran, automatic weapons ban, open carry ban in CA, national debt', 'French Constitution, US Constitution, enumeration of rights, individual rights, subordination to the State', '[Burisma, bribing, Joe Biden]', 'Trump rally, Florida, massive crowd', 'Keene NH, DHS grant, APC, t

In [45]:
test_data['topics'] = generated_topics
print(test_data.shape)
test_data.head()

(104, 4)


,statement,cleaned_tweet,Truthfulness,topics
0,Statistics show thatmore people of color don't...,Dan PatrickIf theyre worried about people of c...,False,"Dan Patrick, drive-in voting, people of color,..."
1,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re..."
2,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon"
3,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ..."
4,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ..."


In [47]:
test_data.to_csv("topic_modelled_test_data.csv", index = False)